# Analysis for Sampling Ternary System

The experiment began shortly before 4pm on Wednesday, October 2nd, 2019. We continued with the same initial solution throughout the rest of the experimental time frame through October 11th, 2019 because of the time required to clean and reload the Parr reactor in the N2 glovebox (~1 day turnaround). Instead, we added cyclopentane to change the recipe by pumping it in with the ISCO pump.

### Concerns
    -Vapor leak changes composition of CO2, current method for accounting for it is heuristic and based on Huikuan's PC-SAFT model (which we were are trying to validate)
    -Vapor leak also changes composition of C5, which I have not accounted for
    -Not getting good matching for high pressure or low temperature, possibly because of drift in accuracy of composition
    -Vapor phase sampling is at a much lower pressure and typically lower temperature (ambient for both) than the headspace in the Parr reactor, resulting in a much lower pressure of the vapor in the GC, so the density must be corrected with a PC-SAFT estimate of the total density (only weight fractions are used from GC, which may be inaccurate in the case of evaporation of a liquid headspace at high pressures)
    -A non-negligible amount of liquid is extracted for sampling (1-2 mL), which can dramatically change the amount of polyol and cyclopentane (and perhaps even CO2) in the Parr reactor. Because we don't know exactly how much is sampled, however, we must rely on a heuristic estimate to predict the amount that has been lost due to sampling at each measurement. So far, this heuristic assumes that only cyclopentane and polyol have been sampled, but perhaps we should include CO2 as well.
    
#### Correcting for CO2 Leak

If I don't correct for leak, the data are obviously off, with the expected density of CO2 being greater than the density in either of the sampled phases. If I use the mass of CO2 predicted under the conditions of each experiment by the PC-SAFT model, I underestimate the CO2 mass resulting in a total density around the measured density of CO2 in the vapor phase (although the density of CO2 in the vapor phase is modified based on the predicted density of the vapor phase assuming the same density as a CO2-C5 mixture in equilibrium with its liquid phase). Right now, I am estimating the leak based on the drop in expected mass predicted by the PC-SAFT model during each pressure step, which assumes a constant leak rate for each pressure step. The result is something in between, and thus, reasonable, but likely still inaccurate (I think it overestimates the density from my observations). Nevertheless, I cannot justify another approximation that would more accurately estimate the amount of CO2 lost through the leak.

In [1]:
import pandas as pd
import numpy as np

import tern
from timedate import TimeDate
import plot
from importlib import reload
reload(tern)

# set user parameters
data_folder = '../../EXPERIMENTS/Dow/Ternary_system_GC/data/'
gc_data_file = '20191002_gc_data.csv'
cond_change_file = '20191002_condition_changes.csv'
save_folder = '../ternary_system_results/'

We begin by opening the data files. For the `df_gc` file, each row corresponds to a measurement by the gas chromatograph (GC). For the `df_cond` file, each row corresponds to a change in condition (injection of cyclopentane or CO2 by ISCO, or venting of Parr reactor through gas-sampling port). We also enter in initial measurements.

In [2]:
# load GC raw data
df_gc = pd.read_csv(data_folder + gc_data_file)
##### initialize experiment
p_parr_offset = 25 # [psi] offset from gauge pressure--Parr has 1 atm N2 gas-->gauge pressure is partial pressure of CO2
# volume of leak upon opening liquid-sampling port after loading in N2 gas glovebox (higher N2 pressure pumped out some liquid)
v_leak = 5 # [mL]
w_poly = 0.51 # weight fraction of polyol in initial C5-polyol mixture [w/w]
rho_poly = 1.02 # density of polyol (VORANOL 2110B) under atmospheric conditions [g/mL]
rho_c5 = 0.751 # density of cyclopentane under atmospheric conditions [g/mL]
purity_c5_a = 0.99073 # cyclopentane from GC lab; measured by Marla Gilbert with GC
m_c5_a = 54.55 # mass of cyclopentane from GC lab[g]
purity_c5_b = 0.98 # cyclopentane ordered from Sigma; read from the bottle
m_c5_b = 27.46 # mass of cyclopentane from Sigma [g]
m_poly_i = 86.59 #initial mass of polyol in Parr reactor before leak [g]
m_poly_i -= w_poly*v_leak*rho_poly # lost some mass while loading [g]
m_c5_i = purity_c5_a*m_c5_a + purity_c5_b*m_c5_b # initial mass of cyclopentane added to Parr reactor [g]
m_c5_i -= (1-w_poly)*v_leak*rho_c5 # lost some mass while loading [g]
m_impurities = (1-purity_c5_a)*m_c5_a + (1-purity_c5_b)*m_c5_b # mass of impurities [g]
V = 1200 # volume of the Parr reactor [mL]; read from manual
T_isco = 21 # temperature of ISCO (same as lab temperature) [C]
timedate_start = TimeDate(date_str='10/02/2019', time_str='15:50:00') # time and date of start of experiment
mixing_rate = 10 # mixing rate of Parr reactor's mixing shaft [RPM]
V_liq_i = m_c5_i / rho_c5 + m_poly_i / rho_poly # initial volume of liquid [mL]
V_liq_sampled = 2*df_gc['sampled liquid'].to_numpy(dtype=bool) # [mL] liquid extracted through HPLIS per sample; *2 b/c 2 samples
V_liq_sampled[25] = 4 # include leak from dip tube when adding C5 the first time [mL]

Next, we analyze the data to estimate the weight fractions, densities, and masses of each component in each phase. These data will be compared to predictions from a PC-SAFT model developed by Dr. Huikuan Chao in the Wang group (Caltech) and fitted with solubility data collected from Dr. Jacob Crosthwaite at Dow Midland and Andy Ylitalo in Prof. Di Maio's lab (U Naples).

In [3]:
reload(tern)
# load condition change data
df_cond = pd.read_csv(data_folder + cond_change_file)
# analyze data
df = tern.analyze_data(df_gc, df_cond, m_poly_i, m_c5_i, V_liq_i, T_isco, timedate_start, p_parr_offset, V_liq_sampled, V)
# save dataframes of results
df.to_csv(save_folder + gc_data_file[:-4] + '_proc_change_rho.csv')
# df_cond.to_csv(data_folder + cond_change_file, index=False)

Finally, we attempt to "correct" the estimate of the mass of CO2 and related variables by estimating the amount that leaked out over time based on how much CO2 is expected to be in the gas phase according to a CO2-C5 binary system at the given temperature and pressure.

In [4]:
reload(tern)
# correct co2 leak
df_corr = tern.correct_co2_leak(df, df_cond, V)
df.to_csv(save_folder + gc_data_file[:-4] + '_proc_corr.csv', index=False)